In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

#### Function that loads a document

In [3]:
def load_document(file):
    from langchain.document_loaders import PyPDFLoader
    print(f'Loading {file}...')
    loader = PyPDFLoader(file)
    data = loader.load()
    return data

#### Load a pdf file data

In [6]:
pip install pypdf

  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/74/a9/5ccde1312650dd03e65350224fea85d9a430c182a01f056599cbb76f7390/pypdf-3.17.0-py3-none-any.whl.metadata
  Using cached pypdf-3.17.0-py3-none-any.whl.metadata (7.5 kB)
Using cached pypdf-3.17.0-py3-none-any.whl (277 kB)

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
data = load_document('./files/us_constitution.pdf')
print(data[0].page_content)

Loading ./files/us_constitution.pdf...
The
United
States
Constitution
W e
the
People
of
the
United
States,
in
Order
to
form
a
more
perfect
Union,
establish
Justice,
insure
domestic
T ranquility ,
provide
for
the
common
defence,
promote
the
general
W elfare,
and
secure
the
Blessings
of
Liberty
to
ourselves
and
our
Posterity ,
do
ordain
and
establish
this
Constitution
for
the
United
States
of
America.
The
Constitutional
Con v ention
Article
I
Section
1:
Congress
All
legislative
Powers
herein
granted
shall
be
vested
in
a
Congress
of
the
United
States,
which
shall
consist
of
a
Senate
and
House
of
Representatives.
Section
2:
The
House
of
Representatives


In [8]:
print(data[0].metadata)

{'source': './files/us_constitution.pdf', 'page': 0}


In [9]:
print(f'You have {len(data)} pages in your pdf')

You have 41 pages in your pdf
